In [16]:
import firebase_admin
from firebase_admin import credentials, firestore

# Initialize app (only once)
try:
    db = firestore.client()
except ValueError:
    # Firebase not initialized, so initialize it
    # If no credentials path provided, use the default one in backend folder
    if not credentials_path:
        # Get the directory where this file is located
        current_dir = os.path.dirname(os.path.abspath(__file__))
        # Go up one level to backend directory
        backend_dir = os.path.dirname(current_dir)
        credentials_path = os.path.join(backend_dir, "databasecreds.json")

    print(f"Looking for credentials at: {credentials_path}")

    if credentials_path and os.path.exists(credentials_path):
        cred = credentials.Certificate(credentials_path)
        firebase_admin.initialize_app(cred)
    else:
        raise FileNotFoundError(
            f"Credentials file not found at {credentials_path}. "
            f"Please ensure databasecreds.json exists in the backend folder."
        )


def delete_docs():
    query = db.collection("users").where("status", "==", "inactive")

    docs = query.stream()

    batch = db.batch()
    count = 0

    for doc in docs:
        batch.delete(doc.reference)
        count += 1

        # Firestore limit: 500 writes per batch
        if count % 500 == 0:
            batch.commit()
            batch = db.batch()

    batch.commit()
    print(f"Deleted {count} documents")


delete_docs()

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.